In [204]:
#! pip install catboost

In [205]:
#! pip install xgboost

In [206]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [207]:
import re
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
#from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

RANDOM_SEED = 42
VAL_SIZE   = 0.20

In [208]:
!ls '../input'}

In [209]:
VERSION    = 4
DIR_TRAIN  = '../input/sfdstcarpriceprediction/' # connected an external dataset to the laptop
#DIR_TRAIN_PARSED  = '../input/final-car-price-prediction-df-parsed-sep-2021/' # connected a parsed dataset from the page `auto.ru`
DIR_TEST   = '../input/sf-dst-car-price-prediction/' 

Из-за долгого времени парсинга, а также крайней нестабильности ситуации на рынке авто, в решении использовали данные из Baseline. Код парсинга представлен ниже

In [210]:
'''
!pip install selenium
!pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import re
import csv
def parsing_old(header, driver, file_name = 'AUTO_RU_DF.csv', page_start = 1, page_end = None):
    done = False
    if page_start == 1:
        params = None
    else:
        params = {'page': str(page_start)}
    pagen = page_start
    while not done:
        with requests.Session() as sess:
            sess.headers.update(header)
            print('Session started')
            old_session = True
            while old_session:
                response = sess.get('https://auto.ru/moskva/cars/used/', params = params, timeout = (15, 20))
                print(response)
                print(response.url)
                print(pagen)
                if response.status_code == 200 :
                    response.encoding = 'gzip'
                    # Теперь создадим объект BeautifulSoup, указывая html парсер    
                    page = BeautifulSoup(response.text, 'html.parser')
                    links = page.find_all('a', class_ = 'Link ListingItemTitle__link')
                    print('found links:   ', len(links))
                    for item in links:
                        url = item['href']
                        print('starting parsing of: ', url)
                        parse_one_car(driver, url)
                        print('parssing done')
                    if pagen == page_end:
                        done = True
                        sess.close()
                    pagen+=1
                    params = {'page': str(pagen)}
                    if pagen % 100 == 0:
                        old_session = False
                        sess.close()
                        print('Session closed')
                    print()
                elif response.status_code == 404:
                    print('404 404 404')
                    done = True
                else:
                    print(response.status_code)
                    print('Some error!')
                    
def parsing(driver, url_default = 'https://auto.ru/moskva/cars/used/', file_name = 'AUTO_RU_DF.csv', page_start = 1, page_end = None):
    #url_default = 'https://auto.ru/moskva/cars/used/'
    done = False
    pagen = page_start
    if page_start == 1:
        url = url_default
    else:
        url = url_default + '?page=' + str(pagen)
    while not done:
        driver.get(url)
        print(url)
        print('page: ',pagen)
        page = BeautifulSoup(driver.page_source, 'html.parser')
        links = page.find_all('a', class_ = 'Link ListingItemTitle__link')
        print('found links:   ', len(links))
        for item in links:
            url = item['href']
            print('starting parsing of: ', url)
            parse_one_car(driver, url)
            print('parssing done')
        if pagen == page_end:
            done = True
            break
        pagen+=1
        url = url = url_default + '?page=' + str(pagen)
                    
def parse_one_car(driver, url, file_name = 'AUTO_RU_DF.csv'):
    driver.get(url)
    page = BeautifulSoup(driver.page_source, 'html.parser')
    item = page_parcer(page, url)
    fields = [item['bodyType'], item['brand'], item['url'], item['color'], None, item['description'],
             item['engineDisplacement'], item['enginePower'], item['equipment_dict'], item['fuelType'], None,
             item['mileage'], None, item['model_info'], item['model_name'], None, None, None, None, item['productionDate'],
             item['sell_id'], None, None, item['vehicleTransmission'], None, item['Владельцы'], item['Владение'],
             item['ПТС'], item['Привод'], item['Руль'], item['Состояние'], item['Таможня'], item['price']]
    with open(file_name, 'a') as f:
        try:
            writer = csv.writer(f)
            writer.writerow(fields)
        except Exception as exc:
            print('File write ERROR: '+ str(exc))
            
def page_parcer(page, url):
    car_dict = {}
    
    try:
        car_dict['bodyType'] = page.find('li', class_ = "CardInfoRow CardInfoRow_bodytype").find('a').text
    except Exception as exc:
        car_dict['bodyType'] = 'Error: ' + str(exc)
    
    try:
        car_dict['productionDate'] = page.find('li', class_ = "CardInfoRow CardInfoRow_year").find('a').text
    except Exception as exc:
        car_dict['productionDate'] = 'Error: ' + str(exc)
    
    try:
        car_dict['mileage'] = page.find('li', class_ = "CardInfoRow CardInfoRow_kmAge").text.replace('\xa0','').replace('Пробег','').replace('км','')
    except Exception as exc:
        car_dict['mileage'] = 'Error: ' + str(exc)
    
    try:
        car_dict['color'] = page.find('li', class_ = "CardInfoRow CardInfoRow_color").find('a').text
    except Exception as exc:
        car_dict['color'] = 'Error: ' + str(exc)
    
    try:
        info = page.find('li', class_="CardInfoRow CardInfoRow_engine").text
        try:
            car_dict['engineDisplacement'] = re.search(r'\d?\d\.\d', info).group() + ' LTR'
        except Exception as exc:
            car_dict['engineDisplacement'] = None
        try:
            car_dict['enginePower'] = re.findall(r'\d?\d?\d\d',info)[0] + ' N12'
        except Exception as exc:
            car_dict['enginePower'] = None
        try:
            car_dict['fuelType'] = re.search(r'\w+$',info).group()
        except Exception as exc:
            car_dict['fuelType'] = None
    except Exception as exc:
        car_dict['engineDisplacement'] = car_dict['enginePower'] = car_dict['fuelType'] = 'Error: ' + str(exc)
    
    try:
        #car_dict['description'] = page.find('div', class_="CardDescriptionHTML").text
        car_dict['description'] = None
    except Exception as exc:
        car_dict['description'] = 'Error: ' + str(exc)
    
    try:
        car_dict['vehicleTransmission'] = page.find('li', class_ = "CardInfoRow CardInfoRow_transmission").find_all('span')[1].text
    except Exception as exc:
        car_dict['vehicleTransmission'] = 'Error: ' + str(exc)
    
    try:
        car_dict['Привод'] = page.find('li', class_ = "CardInfoRow CardInfoRow_drive").find_all('span')[1].text
    except Exception as exc:
        car_dict['Привод'] = 'Error: ' + str(exc)
    
    try:
        car_dict['Руль'] = page.find('li', class_ = "CardInfoRow CardInfoRow_wheel").find_all('span')[1].text
    except Exception as exc:
        car_dict['Руль'] = 'Error: ' + str(exc)
        
    try:
        car_dict['Состояние'] = page.find('li', class_ = "CardInfoRow CardInfoRow_state").find_all('span')[1].text
    except Exception as exc:
        car_dict['Состояние'] = 'Error: ' + str(exc)
        
    try:
        car_dict['Таможня'] = page.find('li', class_ = "CardInfoRow CardInfoRow_customs").find_all('span')[1].text
    except Exception as exc:
        car_dict['Таможня'] = 'Error: ' + str(exc)
        
    try:
        car_dict['Владельцы'] = page.find('li', class_ = "CardInfoRow CardInfoRow_ownersCount").find_all('span')[1].text
    except Exception as exc:
        car_dict['Владельцы'] = 'Error: ' + str(exc)
        
    try:
        car_dict['ПТС'] = page.find('li', class_ = "CardInfoRow CardInfoRow_pts").find_all('span')[1].text
    except Exception as exc:
        car_dict['ПТС'] = 'Error: ' + str(exc)
        
    try:
        car_dict['sell_id'] = page.find('div', class_ = "CardHead__infoItem CardHead__id").text.replace('№ ','')
    except Exception as exc:
        car_dict['sell_id'] = 'Error: ' + str(exc)
        
    try:
        car_dict['Владение'] = page.find('li', class_ = "CardInfoRow CardInfoRow_owningTime").find_all('span')[1].text
    except Exception as exc:
        car_dict['Владение'] = 'Error: ' + str(exc)
        
    try:
        info = page.find_all('div', class_="CardBreadcrumbs__item")
        try:
            car_dict['brand'] = info[2].text
        except Exception as exc:
            car_dict['brand'] = None
        try:
            car_dict['model_name'] = info[3].text
        except Exception as exc:
            car_dict['model_name'] = None
        try:
            car_dict['model_info'] = info[4].text
        except Exception as exc:
            car_dict['model_info'] = None
    except Exception as exc:
        car_dict['brand'] = car_dict['model_name'] = car_dict['model_info'] = 'Error: ' + str(exc)
        
    try:
        car_dict['equipment_dict'] = page.find('li', class_ = "CardInfoRow CardInfoRow_complectationOrEquipmentCount").find_all('span')[1].text
    except Exception as exc:
        car_dict['equipment_dict'] = 'Error: ' + str(exc)
        
    try:
        car_dict['price'] = page.find('span', class_ = "OfferPriceCaption__price").text.replace('\xa0','').replace('₽','')
    except Exception as exc:
        print(exc)
        print(url)
        car_dict['price'] = None
        
    car_dict['url'] = url
        
    return car_dict

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
parsing(driver, 'https://auto.ru/moskva/cars/skoda/used/', page_start = 1, page_end = 99)
parsing(driver, 'https://auto.ru/moskva/cars/lexus/used/', page_start = 1, page_end = 99)
'''

In [211]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [212]:
import warnings
warnings.filterwarnings("ignore")

In [213]:
def IQR(column):
    IQR = df[column].quantile(0.75) - df[column].quantile(0.25)
    minimum = df[column].quantile(0.25) - 1.5*IQR
    maximum = df[column].quantile(0.75) + 1.5*IQR
    return [minimum, maximum]

In [214]:
# убираем ограничение отображения столбцов в датасете
pd.set_option('display.max_columns', None)

In [215]:
train.info()

In [216]:
test.info()

Столбцы, которые мы можем использовать в итоговом датасете:
* bodyType
* brand
* color
* fuelType
* modelDate
* numberOfDoors
* productionDate
* vehicleTransmission
* engineDisplacement
* enginePower
* mileage
* Привод
* Руль
* Владельцы
* ПТС
* Владение

Столбы, которые требуют приведения друг к другу и могут так же быть использованы в итоговом датасете:
* Комплектация в train и complectation_dict в test
* model в train и model_name в test
* start_date в train и parsing_unixtime в test

# Подготовим все столбцы для объединения

In [217]:
columns_in_train = ['bodyType','brand', 'color', 'fuelType', 'modelDate', 'numberOfDoors', 'productionDate', 
                    'vehicleTransmission', 'engineDisplacement', 'enginePower', 'mileage', 
                    'Привод', 'Руль', 'Владельцы', 'ПТС', 'Владение',
                    'start_date', 'model', 'Комплектация', 'price']
columns_in_test = ['bodyType','brand', 'color', 'fuelType', 'modelDate', 'numberOfDoors', 'productionDate', 
                    'vehicleTransmission', 'engineDisplacement', 'enginePower', 'mileage', 
                    'Привод', 'Руль', 'Владельцы', 'ПТС', 'Владение',
                    'parsing_unixtime', 'model_name', 'complectation_dict']

In [218]:
df_train = train[columns_in_train]
df_test = test[columns_in_test]

добавляем столбец цены в test

In [219]:
df_test['price'] = 0

добавляем столбец для идентификации тестовый и учебной выборок

In [220]:
df_train['sample'] = 1
df_test['sample'] = 0

приведем к общему виду start_date и parsing_unixtime

In [221]:
df_train['start_date'].isna().sum()

In [222]:
df_train['start_date'] = pd.to_datetime(df_train['start_date'])

In [223]:
df_test['parsing_unixtime'] = df_test['parsing_unixtime'].apply(datetime.fromtimestamp)

переназовем все столбцы на латинице и эквивалентно

In [224]:
new_columns = ['body_type','brand', 'color', 'fuel_type', 'model_date', 'nod', 'production_date', 
                    'vehicle_transmission', 'engine', 'power', 'mileage', 
                    'drive', 'wheel', 'owners', 'docs', 'owning',
                    'date', 'model', 'complectation', 'price', 'sample']

In [225]:
df_train.columns = df_test.columns = new_columns

In [226]:
df = df_test.append(df_train, sort=False).reset_index(drop=True)

In [227]:
df

## body_type

In [228]:
print('Кол-во вариантов кузова в train', len(df_train['body_type'].unique()))
print('Кол-во вариантов кузова в test', len(df_test['body_type'].unique()))

посмотрим наиболее частые варианты кузова

In [229]:
df_train['body_type'].value_counts()[:15]

In [230]:
df_test['body_type'].value_counts()

In [231]:
# Остаквим ниболее часто встречающиеся варианты
bodyType_list = ['внедорожник', 'седан', 'хэтчбек', 'универсал', 'лифтбек', 'компактвэн',
                 'минивэн', 'пикап', 'купе', 'кабриолет', 'фургон']
def bodyType(item):
    item = str(item).lower()
    for body in bodyType_list:
        if item.find(body) >= 0:
            return (body)
    return 'другое'

In [232]:
df['body_type'] = df['body_type'].apply(bodyType)

In [233]:
df['body_type'].value_counts()

## brand

In [234]:
df['brand'].value_counts()

## color

In [235]:
df['color'].value_counts()

In [236]:
# Приведем к текстовому виду
color_dict = {'040001':"чёрный", 'EE1D19': 'красный', '0000CC':'синий', 'CACECB':'серебристый',
              '007F00':'зелёный', 'FAFBFB':'белый', '97948F':'серый', '22A0F8':'голубой',
              '660099':'пурпурный', '200204':'коричневый', 'C49648':'бежевый',
              'DEA522':'золотистый','4A2197':'фиолетовый', 'FFD600':'жёлтый',
              'FF8649':'оранжевый', 'FFC0CB':'розовый'}

df['color'].replace(color_dict,inplace=True)

In [237]:
df['color'].value_counts()

## fuel_type

In [238]:
df['fuel_type'].value_counts()

In [239]:
# Уберем редкую номенклатуру (объединим в "другое")
df['fuel_type'] = df['fuel_type'].apply(lambda x: x if x in ['бензин', 'дизель'] else 'другое')

In [240]:
df['fuel_type'].value_counts()

## model_date

In [241]:
df['model_date'].value_counts(dropna = False)

In [242]:
# Удалим Nan,приведем значения к int
df.dropna(subset = ['model_date'], inplace = True)
df['model_date'] = df['model_date'].astype(int)

In [243]:
df['model_date'].value_counts(dropna = False)

## nod

In [244]:
df['nod'].value_counts(dropna = False)

In [245]:
df[df['nod'] == 0]

In [246]:
# приведем к целочисленному формату
df['nod'] = df['nod'].astype(int)
# исправим значение 0 на 2 (два сидения у данной модели)
df['nod'] = df['nod'].apply(lambda x: 2 if x == 0 else x)

In [247]:
df['nod'].value_counts()

## production_date

In [248]:
df['production_date'].value_counts()

## vehicle_transmission

In [249]:
df['vehicle_transmission'].value_counts()

In [250]:
# приведем к бинарному виду автомат/механика
transmission_dict = {'AUTOMATIC' : 1, 'MECHANICAL' : 0, 'ROBOT': 1, 'VARIATOR' : 1,
                    'автоматическая' : 1, 'механическая' : 0, 'роботизированная': 1, 'вариатор' : 1}
df['vehicle_transmission'].replace(transmission_dict, inplace = True)

In [251]:
df['vehicle_transmission'].value_counts()

## engine

In [252]:
df['engine'].unique()

Очень много мусорных значений. Т.к. мощность сильно коррелируется с объемом двигателя, оставим в таком виде. Посмотрим на мощность

## power

In [253]:
df['power'].unique()

In [254]:
#В признаке power уберем значение N12
df['power'] = pd.to_numeric(df['power'].replace(regex=[' N12'], value=''))

In [255]:
df['power'] = df['power'].astype(int)

In [256]:
df['power'].value_counts(bins = 10)

## mileage

In [257]:
df['mileage'].value_counts(dropna = False)

## drive

In [258]:
df['drive'].value_counts(dropna = False)

## wheel	

In [259]:
df['wheel'].value_counts(dropna = False)

In [260]:
# приведем к общему виду и сразу сделаем бинарным
wheel_dict = {'LEFT' : 1, 'RIGHT' : 0,
             'Левый' : 1, 'Правый' : 0}
df['wheel'].replace(wheel_dict, inplace = True)

In [261]:
df['wheel'].value_counts()

In [262]:
df.columns

## owners

In [263]:
df['owners'].value_counts(dropna = False)

In [264]:
# заполняем отсутствующие значения
df['owners'] = df['owners'].fillna(3.0)
# переводим в числовой вид (\xa0 - служебный символ "непереносимого" пробела)
owners_dict = {'3 или более' : 3.0, '1\xa0владелец' : 1.0, '2\xa0владельца' : 2.0}
df['owners'].replace(owners_dict, inplace = True)
df['owners'] = df['owners'].astype(int)

In [265]:
df['owners'].value_counts(dropna = False)

## docs

In [266]:
df['docs'].value_counts(dropna = False)

In [267]:
# заполняем отсутствующие значения
df['docs'] = df['docs'].fillna(0)
# переводим в бинарный вид
docs_dict = {'ORIGINAL' : 1, 'Оригинал' : 1, 'DUPLICATE' : 0, 'Дубликат' : 0}
df['docs'].replace(docs_dict, inplace = True)

In [268]:
df['docs'].value_counts(dropna = False)

## owning

In [269]:
df['owning']

если есть хоть какая-то информация это 1, если инфы нет, то 0

In [270]:
df['owning'] = df['owning'].fillna(0)
df['owning'] = df['owning'].apply(lambda x: x if x == 0 else 1)

In [271]:
df['owning'].value_counts()

## date

In [272]:
# уберем время, оставим только дату
df['date'] = df['date'].apply(lambda x: x.date() )

In [273]:
df['date']

## model

In [274]:
df['model'].value_counts()[:15]

In [275]:
# выберем кол-во автомобилей для веделения модели
quantity = 100

In [276]:
# смотрим сколько моделей будет
(df['model'].value_counts()>=quantity).sum()

In [277]:
# создаем маску из True/False для моделей, в который авто больше, чем quantity или нет
models = df['model'].value_counts()>=quantity

In [278]:
models

In [279]:
# оставляем только те, для которых кол-во больше ценза
models = models[models == True]

In [280]:
models

In [281]:
# делаем из series список по индексам
models = list(models.index)

In [282]:
models

In [283]:
# сортируем по длинне названия
models.sort(key=len, reverse=True)

In [284]:
models

In [285]:
def model_name(item):
    for model in models:
        if item.find(model) >= 0:
            return (model)
    return 'OTHER'

In [286]:
df['model'] = df['model'].apply(model_name)

In [287]:
df.sample(5)

## complectation

In [288]:
df['complectation'].value_counts(dropna = False)

In [289]:
df['complectation'] = df['complectation'].fillna("{'id': '0'}")

In [290]:
df['complectation'] = df['complectation'].apply(lambda x: 0 if x == "{'id': '0'}" else 1)

In [291]:
df['complectation'].value_counts(dropna = False)

## price

In [292]:
# целевая переменная, все что не заполнено удаляем
df

In [293]:
df['price'].isna().sum()

In [294]:
df.dropna(subset = ['price'], inplace = True)

In [295]:
# удалим так же неудачный столбец с объемом двигателя
df.drop('engine', axis = 1, inplace = True)

In [296]:
df

-----------------------------------

# Feature engeneering

## 1) Редкость цвета

создадим новый столбец с редкостью цвета, если цвет не входит в шесть самых популярных, считаем его редким

In [297]:
df['color'].value_counts()

In [298]:
rare_dict = list(df['color'].value_counts()[6:].index)

In [299]:
df['color_rarety'] = df['color'].apply(lambda x: 1 if x in rare_dict else 0)

In [300]:
df.sample(10)

## 2) Раритет

 если марка автомобиля была разработана до 1980 года, считаем машину раритетом

In [301]:
df['rarity'] = df['model_date'].apply(lambda x: 1 if x < 1980 else 0)

In [302]:
df['rarity'].value_counts()

## 3) "Корзины" пробега 

уменьшим разброс пробега, поделив на 10000 км и ограничив 400000 (все что выше отсекаем как 400000)

In [303]:
df['mileage_bins'] = df['mileage'].apply(lambda x: int(float(x))//10000)

In [304]:
df

In [305]:
df['mileage_bins'].value_counts(bins = 10)

In [306]:
df['mileage_bins'] = df['mileage_bins'].apply(lambda x: x if x<40 else 40)

In [307]:
df['mileage_bins'].value_counts(bins = 5)

## 4) Дата парсинга

предположительно, дата парсинга данных в train 09.09.2020 (по названию файла), а что такое start_date остается загадкой
создадим новый столбец с датой парсинга (для test оставим date, а для train 09.09.2020)

In [308]:
df.info()

In [309]:
df['parsing_date'] = df['date']

In [310]:
def parsing_date(item):
    if item['sample'] == 1:
        item['parsing_date'] = '2020-09-09'
    else:
        item['parsing_date'] = item['date']
    return item

In [311]:
df = df.apply(parsing_date, axis = 1)

In [312]:
df['parsing_date'] = pd.to_datetime(df['parsing_date'])

## 5) Возраст модели

сделаем два варианта: от date и от parsing_date

In [313]:
df['date'] = pd.to_datetime(df['date'])
df['model_age_date'] = df['date'].apply(lambda x: x.year)
df['model_age_parsing'] = df['parsing_date'].apply(lambda x: x.year)

In [314]:
df['model_age_date'] = df['model_age_date'] - df['model_date']
df['model_age_parsing'] = df['model_age_parsing'] - df['model_date']

## 6) Возраст автомобиля

сделаем два варианта: от date и от parsing_date

In [315]:
df['production_age_date'] = df['date'].apply(lambda x: x.year)
df['production_age_parsing'] = df['parsing_date'].apply(lambda x: x.year)

In [316]:
df['production_age_date'] = df['production_age_date'] - df['production_date']
df['production_age_parsing'] = df['production_age_parsing'] - df['production_date']

## 7) Пробег в год

сделаем два варианта: от date и от parsing_date

In [317]:
df['mpe_date'] = (df['mileage'] / df['production_age_date']) / 1000
df['mpe_parsing'] = (df['mileage'] / df['production_age_parsing']) / 1000

In [318]:
df['mpe_date'] = df['mpe_date'].replace(np.inf, np.nan)
df['mpe_parsing'] = df['mpe_parsing'].replace(np.inf, np.nan)

In [319]:
df['mpe_date'].fillna(0, inplace = True)
df['mpe_parsing'].fillna(0, inplace = True)

In [320]:
df['mpe_date'] = df['mpe_date'].apply(lambda x: int(x))
df['mpe_parsing'] = df['mpe_parsing'].astype(int)

# Группировка признаков, анализ

In [321]:
df.columns

In [322]:
num_cols = ['model_date', 'production_date', 'power', 'mileage', 'owners', 'mileage_bins', 'model_age_date',
            'model_age_parsing', 'production_age_date', 'production_age_parsing', 'mpe_date', 'mpe_parsing']
cat_cols = ['body_type', 'brand', 'color', 'fuel_type', 'nod', 'drive', 'model']
bin_cols = ['vehicle_transmission', 'wheel', 'docs', 'owning', 'complectation', 'color_rarety', 'rarity']

## Числовые признаки

In [323]:
# Работа с выбросами
# Построим боксплоты
for i in num_cols:
    df[[i]].boxplot()
    plt.show()

In [324]:
df['price'].hist()

In [325]:
df[['price']][df['sample'] == 1].describe()

In [326]:
#df['price'] = df['price'].apply(lambda x: None if x > 10000000 else x)

In [327]:
#df['price'].isna().sum()

In [328]:
#df.dropna(subset=['price'],inplace=True)

In [329]:
# Посмотрим на распределение числовых признаков
for i in num_cols:
    plt.figure()
    sns.displot(df[i], kde = False, rug=False)
    plt.title(i)
    plt.show()

In [330]:
# Признак Владельцы логарифмировать не имеет смысла. Остальные прогогарифмируем
#for i in num_cols:
#    if i == 'Владельцы': continue
#    df[i] = df[i].apply(lambda x: np.log(abs(x)+1))
#    plt.figure()
#    sns.distplot(df[i][df[i] > 0], kde = False, rug=False)
#    plt.title(i)
#    plt.show()

In [331]:
# Посмотрим влияние числовых переменных на целевую
imp_num = pd.Series(f_classif(df[df['sample'] == 1][num_cols], df[df['sample'] == 1]['price'].values)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [332]:
# Посмотрим матрицу корреляций числовых признаков:
plt.subplots(figsize=(15, 15))
sns.heatmap(df[df['sample'] == 1][num_cols + ['price']].corr(), square=True,
              annot=True, fmt=".3f", linewidths=0.1, cmap="RdBu")
plt.show()

Удалим сильно скоррелированные признаки, которые меньше влияют на цену авто

In [333]:
num_cols = ['power', 'owners', 'mileage_bins', 'model_age_parsing', 'production_age_parsing', 'mpe_parsing']

## Категориальные признаки

In [334]:
#Преобразование категориальных признаков
df_CB = df.copy(deep=True)

for item in cat_cols:
    df_CB[item] = df_CB[item].astype('category')
    df_CB[item] = df_CB[item].cat.codes

In [335]:
# Для оценки значимости категориальных переменных будем использовать функцию mutual_info_classif из библиотеки sklearn
imp_cat = pd.Series(mutual_info_classif(df_CB[df_CB['sample'] == 1][bin_cols + cat_cols], df_CB[df_CB['sample'] == 1]['price'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Удалим признаки, маловлияющие на цену авто

In [336]:
bin_cols = ['vehicle_transmission', 'docs', 'owning', 'complectation', 'color_rarety']

# Подготовка данных к обучению

In [337]:
df = df[num_cols + cat_cols + bin_cols + ['sample', 'price']]

In [338]:
df = pd.get_dummies(df, columns = bin_cols + cat_cols)

In [339]:
df

In [340]:
# Теперь выделим тестовую часть
train = df.query('sample == 1').drop(['sample'], axis=1)
test = df.query('sample == 0').drop(['sample', 'price'], axis=1)

y = train['price'].values  # наш таргет
X = train.drop(['price'], axis=1)

In [341]:
# Стандартизация числовых переменных
scaler = StandardScaler()
scaler.fit(X[num_cols].values)

X_num = scaler.transform(X[num_cols].values)

In [342]:
rest_cols = list(X.columns)
for col in num_cols:
    rest_cols.remove(col)

In [343]:
X = np.hstack([X_num, X[rest_cols].values])

In [344]:
X.shape

In [345]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# ML

Из-за долгого времени подбора параметров моделей, подбор параметров осущевствлялся не в этом ноутбуке

In [346]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

## 1. CatBoost

In [347]:
'''
model = CatBoostRegressor(iterations = 5000,
                          random_seed = 42,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         )
'''

In [348]:
# оцениваем точность
#predict = model.predict(X_test)
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

In [349]:
'''
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         )
'''

In [350]:
#predict_test = np.exp(model.predict(X_test))
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

## 2. Стэкинг

In [351]:
'''
estimators = [
    ('rf', ExtraTreesRegressor(random_state=42, n_jobs=-1, verbose=1)),
    ('xgb', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.05, max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000, random_state=42))
]

sr_log = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)

sr_log.fit(X_train, np.log(y_train))

y_pred = np.exp(sr_log.predict(X_test))

print(f"The MAPE mertics of the default StackingRegressor model: {(mape(y_test, y_pred) * 100):0.2f}%.")
'''

## 3. SGDRegressor

In [352]:
'''
linear_regression_model = SGDRegressor(tol=.0001, eta0=.01)
linear_regression_model.fit(X_train, np.log(y_train))
predictions = np.exp(linear_regression_model.predict(X_test))
print( f"Точность модели по метрике MAPE с логтаргетом: {(mape(y_test, predictions))*100:0.2f}%")
'''

## 4. XGBRegressor

In [353]:

gb = xgb.XGBRegressor( objective='reg:squarederror', colsample_bytree=0.5, 
                      learning_rate=0.05, max_depth=12, alpha=1, n_estimators=1000,random_state=42, n_jobs=-1)

gb.fit(X_train, np.log(y_train))
predict_xgb = np.exp(gb.predict(X_test))

print( f"Точность модели по метрике MAPE с логтаргетом: {(mape(y_test, predict_xgb))*100:0.2f}%")


## 5. RandomForest

In [354]:
'''
rf = ExtraTreesRegressor(n_estimators=300, random_state=42, n_jobs=-1, bootstrap=True, verbose=1)
rf.fit(X_train, np.log(y_train))
pred_rf = np.exp(rf.predict(X_test))

print(f"Точность модели по метрике MAPE: {(mape(y_test, pred_rf))*100:0.2f}%")
'''

## Выбираем лучшую модель

Выбираем gb, т.к. время работы стекинга очень большое, а результат отличается на 0,02

Заново обучаем ее на всей выборке sample

In [355]:
gb.fit(X, np.log(y))

In [356]:
VERSION = 5

In [357]:
X_sub = test
X_sub_num = scaler.transform(X_sub[num_cols].values)
X_sub = np.hstack([X_sub_num, X_sub[rest_cols].values])
print(X_sub.shape)

In [358]:
predict_submission = np.exp(gb.predict(X_sub))

## Попробуем дополнительно ввести коррекцию по курсу валют

In [359]:
cur = pd.read_csv(DIR_TRAIN+'usd_quotes.csv', sep = ';') # датасет с курсом доллара к рублю
cur.info()

In [360]:
cur.head(15)

In [361]:
cur['updated'] = pd.to_datetime(cur['updated']) # переводим в формат даты
cur['year'] = cur['updated'].apply(lambda x: x.year) # создаем столбец с годом
cur['month'] = cur['updated'].apply(lambda x: x.month) # создаем столбец с месяцем
# получим вложенный словарь со среднем курсом валюты за месяц 
rate_dict = {}
for year in cur['year'].unique():
    rate_dict[year] = {} # создаем словарь в словаре
    for month in cur['month'][cur['year'] == year].unique():
        rate_dict[year][month] = cur['price'][(cur['year'] == year) & (cur['month'] == month)].mean()

In [362]:
sample_submission['price'] = predict_submission
sample_submission

In [363]:
# выведем даты временного интервала в который были спарсены данные тестовой выборки
print(df_test['date'].max())
print(df_test['date'].min())

Судя по названию файла с основным датасетом данные получены 09.09.2020, а данные из тестового в конце 10.2020.
Для корректировки цены используем среднюю котировку за 9 месяц и за 10.

In [364]:
sample_submission['price'] = (sample_submission['price']/rate_dict[2020][9])*rate_dict[2020][10]

In [365]:
#sample_submission['price'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)